In [1]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
sys.path.append( '/home/cbisot/pycode/MscThesis/amftrack/pipeline/functions')

from amftrack.notebooks.analysis.util import *
from amftrack.pipeline.paths.directory import path_code, directory_scratch, directory_project
from amftrack.notebooks.analysis.data_info import *
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.ndimage import gaussian_filter
from scipy import ndimage
from scipy.optimize import curve_fit

In [2]:
exp = get_exp((39,269,329),directory_project)
# exp = get_exp((39,7,42),directory_project)

begin = 2020-11-23 15:01:00 
  end = 2020-11-26 03:02:00


In [16]:
def criter(max_growth,length):
    return(a*length+b*max_growth>=2.5 and max_growth>=50)

In [17]:
RH, BAS, max_speeds, total_growths, widths, lengths, branch_frequ,select_hyph = get_rh_bas(exp,criter)

failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure 

In [18]:
lengths = [get_length(exp,t) for t in range(exp.ts)]

In [19]:
times = [get_time(exp,0,t) for t in range(exp.ts)]

In [20]:
bas_roots = [hyph.root.label for hyph in BAS]
rh_roots = [hyph.root.label for hyph in RH]
rh_length = 0
count_rh = 0
count_bas = 0
bas_length = 0
for rh in RH:
    length = rh.get_length_um(rh.ts[-1])
    nodes = rh.get_nodes_within(rh.ts[-1])[0]
    for node in nodes[1:-1]:
        if node in rh_roots:
            count_rh+=1
    rh_length += length
for rh in BAS:
    if len(rh.ts)>0:
        length = rh.get_length_um(rh.ts[-1])
        nodes = rh.get_nodes_within(rh.ts[-1])[0]
        for node in nodes[1:-1]:
            if node in rh_roots:
                count_bas+=1
        bas_length += length
    

In [21]:
plt.close('all')
t0s = {t:0 for t in range(exp.ts)}
for rh in RH:
    for t in rh.ts:
        t0s[t]+=1
    
# for rh in BAS:
#     for t in rh.ts:
#         t0s[t]+=1
nrh = list(t0s.values())
X = np.array(times[:]).reshape(-1,1)
y = np.log(np.array(nrh[:]))
reg = LinearRegression().fit(X, y)
# nrh= np.cumsum(list(t0s.values()))
# fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
# ax.scatter(times,np.array(nrh)/nrh[0])
# ax.plot(times,np.exp(times*(reg.coef_+0.002)))
fig = plt.figure()
ax = fig.add_subplot(111)
end = 40

def func2(t,alpha,C,d):
    return(C*(np.exp(alpha*(t)))+d)
popt1, pcov = curve_fit(func2,times[:end],np.array(nrh)[:end],bounds = ([0,0,-np.inf],3*[np.inf]),p0=[1,1,0])
# popt2, pcov = curve_fit(func2, times, total_anastomosis_theory,bounds = ([0,0,-np.inf],3*[np.inf]),p0=[1,1,0])

ax.scatter(times,np.array(nrh), label = 'Total number of apexes')
ax.plot(times,func2(np.array(times),*popt1),label = f'exponential fit : alpha= {"{:.2f}".format(popt1[0]*24)}.day-1',color='blue')
ax.set_xlabel('lapse (h)')
ax.set_ylabel('P(t>lapse)')
plt.legend()

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
def func(t,alpha,C):
    return(C*(np.exp(alpha*(t))))
end = 30
begin = 0
coefs = np.polyfit(times[begin:end],lengths[begin:end],2)
popt0, pcov = curve_fit(func, times[begin:end], lengths[begin:end],bounds = ([0,0],2*[np.inf]),p0=[1,1])
p = np.poly1d(coefs)

In [13]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
ax.scatter(times,np.array(lengths)/1000,label = 'data')
ax.plot(times,func(np.array(times),*popt0)/1000,label = f'exponential fit : {popt0[0]*24}')
ax.plot(times,p(np.array(times))/1000, label= 'quadratic fit')
ax.set_xlim(times[begin],times[end])
ax.set_ylim(lengths[begin]/1000,lengths[end]/1000)
ax.set_xlabel('time (h)')
ax.set_ylabel('network length (mm)')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
coefs,popt0

(array([2.84005968e+02, 1.21617807e+02, 2.00305164e+05]),
 array([3.33707669e-02, 1.70360071e+05]))

In [39]:
coefs,popt0

(array([  170.61632157, -3616.95220827, 65812.40371117]),
 array([3.15512382e-02, 6.49139360e+04]))

In [42]:
coefs,popt0

(array([  170.61632157, -3616.95220827, 65812.40371117]),
 array([4.86588808e-02, 2.95647542e+04]))

In [25]:
plt.close('all')
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
end = -1
X = np.array(times[:end]).reshape(-1,1)
y = np.log(np.array(lengths[:end]))
reg = LinearRegression().fit(X, y)
coefa=reg.coef_
ax.scatter(times,np.array(lengths)/lengths[0])
ax.plot(times,np.exp(np.array(times)*coefa),label = f'total_length {"{:.2f}".format(float(coefa)*24)}day-1')
ax.set_xlim(0,80)
ax.set_ylim(0,20)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
reg.coef_,np.exp(reg.intercept_)

(array([0.03322134]), 171297.01698299826)

In [33]:
np.pi*v**2*gamma/(2*c)

0.0029344050084903514

In [34]:
2/(np.pi*gamma*v**2/(c*alpha)-alpha)

array([13.94345301])

In [35]:
np.pi*gamma*v**2/(c*alpha),alpha

(array([0.17665783]), array([0.03322134]))

In [43]:
a

array([4.23580305e-08])

In [45]:
(np.sqrt(delta)-b)

array([1005.3096415])

In [46]:
a=c*alpha**3/6
b = (c*alpha**2-gamma*np.pi*v**2)/2
c = c*alpha
delta = b**2-4*a*c
delta,b,np.sqrt(delta)
(np.sqrt(delta)-b)/(2*a),(-np.sqrt(delta)-b)/(2*a)

(array([3.5720452e+11]), array([0.]))

In [32]:
gamma = 0.008 #um-1
c = np.exp(reg.intercept_) #um
alpha = reg.coef_ #hour
v = 200 #um.h-1
def f(t):
    return(np.exp(alpha*t)-np.pi*v**2*t**2/(2*gamma*c))

In [29]:
plt.close('all')
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
x = np.linspace(0,200,50)
ax.plot(x,f(x))

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
def get_density_maps(exp,t,compress,kern_sizes):
    skeletons = [sparse.csr_matrix(skel) for skel in exp.skeletons]
    window=compress
    densities=np.zeros((skeletons[t].shape[0]//compress,skeletons[t].shape[1]//compress),dtype=np.float)
    for xx in range(skeletons[t].shape[0]//compress):
        for yy in range(skeletons[t].shape[1]//compress):
            x = xx*compress
            y = yy*compress
            skeleton=skeletons[t][x-window:x+window,y-window:y+window]
            density = skeleton.count_nonzero()/((window*1.725)**2)
            densities[xx,yy]=density
    results = {}
    for kern_size in kern_sizes:
        density_filtered = gaussian_filter(densities,kern_size)
        sx = ndimage.sobel(density_filtered ,axis=0,mode='constant')
        sy = ndimage.sobel(density_filtered ,axis=1,mode='constant')
        sobel=np.hypot(sx,sy)
        results[kern_size] = density_filtered,sx,sy,sobel
    return(results)

In [ ]:
resu

In [16]:
result = get_density_maps(exp,-1,100,[5,10])
density_filtered,sx,sy,sobel = result[5]

In [30]:
plt.close('all')
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
ax.imshow(densities)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
plt.close('all')
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
ax.imshow(density_filtered)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
sx = ndimage.sobel(density_filtered ,axis=0,mode='constant')
sy = ndimage.sobel(density_filtered ,axis=1,mode='constant')
sobel=np.hypot(sx,sy)
plt.close('all')
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
ax.imshow(sobel)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
exp.plot_raw(-1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
skeletons[-1].shape

(30000, 60000)

In [120]:
coefa*24

array([1.47797054])

In [86]:
total_growth= [274.4182939842498,
 264.75066857923247,
 130.14008759834883,
 314.0507474032412,
 113.7477630475859,
 1161.622784505229,
 1745.0546022253006,
 2726.9218066428225,
 2871.778197123677,
 4383.75894167157,
 4214.542457713185,
 3085.5196044227305,
 3814.118401873534,
 3850.501563058079,
 2799.2820758732155,
 644.4555313848887,
 1955.058800564103,
 488.4187953798312,
 5437.132269415376,
 6029.556668562571,
 309.74440034631823,
 2827.3429474183235,
 7237.124025346528,
 6157.3855798877885]

In [6]:
plt.close('all')
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
end = -1
X = np.array(times[:end]).reshape(-1,1)
y = np.log(np.array(lengths[:end]))
reg = LinearRegression().fit(X, y)
coefa=reg.coef_
ax.scatter(times,np.array(lengths)/lengths[0])
ax.plot(times,np.exp(np.array(times)*coefa),label = f'total_length {"{:.2f}".format(float(coefa)*24)}day-1')
t0s = {t:0 for t in range(exp.ts)}
t0s_l = {t:0 for t in range(exp.ts)}
for rh in RH:
    for t in rh.ts:
        t0s[t]+=1
        t0s_l[t]+=rh.get_length_um(t)
# nrh= np.cumsum(list(t0s.values()))
nrh = list(t0s_l.values())
X = np.array(times[:end]).reshape(-1,1)
y = np.log(np.array(nrh[:end])+1)
reg = LinearRegression().fit(X, y)
ax.scatter(times,np.array(nrh)/nrh[0])
coefb=reg.coef_
ax.plot(times,np.exp(times*reg.coef_),label=f'length rh {"{:.2f}".format(float(coefb)*24)}day-1')
nrh = list(t0s.values())
X = np.array(times[:end]).reshape(-1,1)
y = np.log(np.array(nrh[:end])+1)
reg = LinearRegression().fit(X, y)
coefc=reg.coef_
ax.scatter(times,np.array(nrh)/nrh[0])
ax.plot(times,np.exp(times*reg.coef_),label=f'number rh {"{:.2f}".format(float(coefc)*24)}day-1')
# t0s = {t:0 for t in range(exp.ts)}
# t0s_l = {t:0 for t in range(exp.ts)}
# for rh in RH:
#     for t in rh.ts:
#         t0s[t]+=1
#         t0s_l[t]+=rh.get_length_um(t)
# for rh in BAS:
#     for t in rh.ts:
#         t0s[t]+=1
#         t0s_l[t]+=rh.get_length_um(t)
# nrh = list(t0s_l.values())
# popt0, pcov = curve_fit(func, times, nrh,bounds = ([0,0],2*[np.inf]),p0=[1,1])
# coef = popt0[-1]

# ax.scatter(times,np.array(nrh)/nrh[0])
# ax.plot(times,np.exp(np.array(times)*coef),label='length rh and bas')
# nrh = list(t0s.values())
# popt0, pcov = curve_fit(func, times, nrh,bounds = ([0,0],2*[np.inf]),p0=[1,1])
# coef = popt0[-1]

# nrh= np.cumsum(list(t0s.values()))
# fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
# ax.scatter(times,np.array(nrh)/nrh[0])
# ax.plot(times,np.exp(np.array(times)*coef),label='number rh and bas')
# X = np.array(times[:end-2]).reshape(-1,1)
# y = np.log(np.array(total_growth[:end]))
# reg = LinearRegression().fit(X, y)
# coefd = reg.coef_
# ax.scatter(times[:-2],np.array(total_growth)/total_growth[0])
# ax.plot(times,np.exp(times*reg.coef_),label=f'total growth of RH {"{:.2f}".format(float(coefd)*24)}day-1')
ax.set_xlabel('time (h)')
ax.set_ylabel('normalised increase')
plt.legend()

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [97]:
ax.set_ylim(0,1000)


(0, 1000)

In [90]:
len(times[:end]),len(total_growth)

(35, 24)

In [8]:
branch_dist = rh_length/count_rh
mean_speed = np.mean([speed for speed in speeds if speed>25])
branch_dist,mean_speed,mean_speed/branch_dist*24*np.log(2)

(3834.2808433161085, 194.5779672145838, 0.8442021326151778)

In [7]:
window = 800
skeletons = [sparse.csr_matrix(skel) for skel in exp.skeletons]
angles = []
curvatures = []
densities = []
growths = []
speeds = []
tortuosities = []
hyphs = []
ts = []
print(f'There is {len(RH)} RH')
for hyph in RH:
    for i,t in enumerate(hyph.ts[:-1]):
        tp1=hyph.ts[i+1]
        segs,nodes = get_pixel_growth_and_new_children(hyph,t,tp1)
        speed = np.sum([get_length_um(seg) for seg in segs])/get_time(exp,t,tp1)
        total_growth = speed * get_time(exp,t,tp1)
        curve = [pixel for seg in segs for pixel in seg]
        pos = hyph.end.pos(t)
        x,y = pos[0],pos[1]
        straight_distance = np.linalg.norm(hyph.end.pos(t)-hyph.end.pos(tp1))
        skeleton=skeletons[t][x-window:x+window,y-window:y+window]
        density = skeleton.count_nonzero()/(window*1.725)
        curve_array = np.array(curve)
        res = 50
        index = min(res,curve_array.shape[0]-1)
        vec1 = curve_array [index]-curve_array [0]
        vec2 = curve_array [-1]-curve_array [-index-1]
        if np.linalg.norm(vec1)>0 and np.linalg.norm(vec2)>0:
            begin = vec1/np.linalg.norm(vec1)
            end = vec2/np.linalg.norm(vec2)
            dot_product = min(np.dot(begin, end),1)
            if (begin[1] * end[0] - end[1] * begin[0] >= 0):  # determinant
                angle = -np.arccos(dot_product) / (2 * np.pi) * 360
            else:
                angle = np.arccos(dot_product) / (2 * np.pi) * 360
            inv_tortuosity = (straight_distance*1.725)/total_growth
            if hyph.end.degree(tp1)<3 and inv_tortuosity>0.8 and inv_tortuosity<1.1:
                if np.isnan((angle/total_growth)):
                    print(angle,total_growth,dot_product)
                angles.append(angle)
                curvatures.append(angle/total_growth)
                densities.append(density)
                growths.append(total_growth)
                speeds.append(speed)
                tortuosities.append(inv_tortuosity)
                ts.append(get_time(exp,0,t))
                hyphs.append(hyph.end.label)

There is 308 RH


In [123]:
reg.coef_

array([0.03663874])

In [105]:
end = -10
X = np.array(times[:end]).reshape(-1,1)
reg = LinearRegression().fit(np.array(times[:end]).reshape(-1,1), np.log(np.array(lengths[:end])))
reg.coef_*24

array([0.79399848])

In [22]:
reg.coef_*24

array([0.79628228])

In [20]:
X = np.array(times).reshape(-1,1)
X

array([[ 0.        ],
       [ 1.        ],
       [ 2.        ],
       [ 3.        ],
       [ 3.98333333],
       [ 4.98333333],
       [ 5.98333333],
       [ 6.98333333],
       [ 7.98333333],
       [ 8.98333333],
       [ 9.98333333],
       [11.        ],
       [12.        ],
       [13.        ],
       [14.        ],
       [15.        ],
       [16.        ],
       [17.        ],
       [18.        ],
       [19.        ],
       [20.        ],
       [21.01666667],
       [22.01666667],
       [23.01666667],
       [24.01666667],
       [25.01666667],
       [26.01666667],
       [27.01666667],
       [28.01666667],
       [29.01666667],
       [30.01666667],
       [31.01666667],
       [32.01666667],
       [33.01666667],
       [34.01666667],
       [35.01666667],
       [36.01666667],
       [37.01666667],
       [38.01666667],
       [39.01666667],
       [40.01666667],
       [41.01666667],
       [42.01666667],
       [43.01666667],
       [44.01666667],
       [45

In [32]:
def estimate_rh_freq_mult(inst,samples,back,criter, path):
    bas_frequ = []
    t0 = inst[2]-inst[1]-back
    exp = get_exp(inst,path)
    RH, BAS, max_speeds, total_growths, widths, lengths, branch_frequ,select_hyph = get_rh_bas(exp,criter)
    bas_roots = [hyph.root for hyph in BAS]
    rh_roots = [hyph.root for hyph in RH]
    for k in range(samples):
        node1 = Node(choice(list(exp.nx_graph[t0].nodes)),exp)
        node2 = Node(choice(list(exp.nx_graph[t0].nodes)),exp)
        if np.linalg.norm(node1.pos(t0)-node2.pos(t0))>=5000 and nx.algorithms.shortest_pipeline.paths.generic.has_path(exp.nx_graph[t0],node1.label,node2.label):
            nodes = nx.shortest_path(exp.nx_graph[t0], source = node1.label, target = node2.label)
#             exp.plot([t],[nodes])
            bass=[]
            for node in nodes:
                if exp.get_node(node) in  rh_roots:
                    bass.append(node)
        #             print(hyph)
        #             if hyph.ts[0] in rh.ts:
        #                 print(hyph,hyph_info[hyph],hyph in BAS)
        #                 hyph.end.show_source_image(hyph.ts[-1],hyph.ts[-1])
            print(len(bass)/get_length_um_node_list(nodes,exp,t0)*10000)
            bas_frequ.append(len(bass)/get_length_um_node_list(nodes,exp,t0)*10000)
            print(bas_frequ)
#         pickle.dump(bas_frequ, open(f'{path_code}/MscThesis/Results/bas_{inst}.pick', "wb"))
    return(bas_frequ)

In [33]:
bas_frequ = estimate_rh_freq_mult((39,269,329),10,0,criter, directory_project)

begin = 2020-11-23 15:01:00 
  end = 2020-11-26 03:02:00
4.191771057526126
[4.191771057526126]
2.7759026885992615
[4.191771057526126, 2.7759026885992615]
2.306781045135673
[4.191771057526126, 2.7759026885992615, 2.306781045135673]
2.496018946267994
[4.191771057526126, 2.7759026885992615, 2.306781045135673, 2.496018946267994]
5.405483136650545
[4.191771057526126, 2.7759026885992615, 2.306781045135673, 2.496018946267994, 5.405483136650545]
3.682288072807331
[4.191771057526126, 2.7759026885992615, 2.306781045135673, 2.496018946267994, 5.405483136650545, 3.682288072807331]
1.985722546221831
[4.191771057526126, 2.7759026885992615, 2.306781045135673, 2.496018946267994, 5.405483136650545, 3.682288072807331, 1.985722546221831]


In [40]:
np.mean(bas_frequ)/10000

0.0004098985079133463

In [39]:
len(RH)/lengths[-1]*np.mean([speed for speed in speeds if speed>25])*24

1.1747647373447787

In [38]:
np.mean([speed for speed in speeds if speed>25])*np.mean(bas_frequ)/10000*24*np.log(2)

1.0563403230678294

In [47]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
ax.hist([frequ for frequ in rh_frequ if frequ>0],20)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 9., 15.,  7.,  4.,  5.,  6., 10.,  6.,  1.,  2.,  1.,  1.,  3.,
         2.,  1.,  0.,  0.,  1.,  0.,  1.]),
 array([8.10016953e-05, 1.78435621e-04, 2.75869548e-04, 3.73303474e-04,
        4.70737400e-04, 5.68171326e-04, 6.65605252e-04, 7.63039178e-04,
        8.60473105e-04, 9.57907031e-04, 1.05534096e-03, 1.15277488e-03,
        1.25020881e-03, 1.34764274e-03, 1.44507666e-03, 1.54251059e-03,
        1.63994451e-03, 1.73737844e-03, 1.83481237e-03, 1.93224629e-03,
        2.02968022e-03]),
 <a list of 20 Patch objects>)

In [48]:
rh_frequ_better = [frequ for frequ in rh_frequ if frequ>0]

In [49]:
np.mean(rh_frequ_better)*np.mean([speed for speed in speeds if speed>25])*24

2.7516414060891012

In [50]:
np.mean(rh_frequ_better)

0.0005892328247384389

In [44]:
np.mean(bas_frequ)/10000

0.0004098985079133463

In [ ]:
rh_length/lengths[-1]

In [77]:
(bas_length+rh_length)/lengths[-1]

1.3318809414965531

In [53]:
count

189

0.8442021326151778

In [58]:
len(RH)/lengths[-1]*np.mean([speed for speed in speeds if speed>25])*24

1.1747647373447787

In [55]:
rh_length/lengths[-1]

0.5918900042207608

In [110]:
t0s = {t:0 for t in range(exp.ts)}
for rh in RH:
    for t in rh.ts:
        t0s[t]+=1
# nrh= np.cumsum(list(t0s.values()))
nrh = list(t0s.values())

In [111]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
ax.scatter(times,nrh)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [88]:
X = np.array(times[:-4]).reshape(-1,1)
y = np.log(np.array(nrh[:-4]))
reg = LinearRegression().fit(X, y)
reg.coef_*24,reg.score(X, y)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [82]:
t0s = {t:0 for t in range(exp.ts)}
for rh in RH:
    for t in rh.ts:
        t0s[t]+=1
    
for rh in BAS:
    for t in rh.ts:
        t0s[t]+=1
nrh = list(t0s.values())
# nrh= np.cumsum(list(t0s.values()))
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
ax.scatter(times,nrh)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [84]:
X = np.array(times[:-4]).reshape(-1,1)
y = np.log(np.array(nrh[:-4]))
reg = LinearRegression().fit(X, y)
reg.coef_*24,reg.score(X, y)

(array([0.87420532]), 0.9923801052736214)

In [72]:
plt.close('all')
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
ax.scatter(ts,speeds)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …